<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Building Agentic AI Applications with LLMs</h1>
<h2><b>Assessment:</b> Creating A Basic Researching Agent</h2>
<br>

**Welcome To The Assessment!** We hope you're ready to apply some of the skills you've learned so far towards building something you've probably seen floating around; a "researching" chatbot. The overall idea should be pretty familiar:

- **The chatbot should look at your question and look around the internet for some resources.**
- **Based on those resources, the chatbot should make an educated guess based on its retrieved information.**

This is implemented often in conjunction with LLM interfaces like ChatGPT and Perplexity, and various open-source efforts have cropped up to simplify the process. With that being said, they usually don't rely on the likes of 8B models due to the finicky nature of routing them properly. As such, we will merely be testing you on your ability to implement the following primitives: 
- **A structured output interface to produce a parseable list.**
- **A function to search for web snippets and filter out the most relevant results.**
- **A mechanism for accumulating message beyond the control of the user.**
- **Some basic prompt engineering artifacts.**

Of note, there are many extensions which you should be able to imagine at this point. Perhaps we could have a requerying mechanism somewhere? Or maybe either the user or the agent could criticize and remove entries from the history? Long-term memory does sound appealing, after all. However, we will be focusing on just our simple features as required for two key reasons:
- **First, we really don't want to force you to do more engineering than you have to.** Frameworks like LangGraph may have many levers and introduce new primitives very quickly in an attempt to simplify the interface, so any overengineering we do now may become deprecated by the time you're reading this with some simpler off-the-shelf options.
- **Secondly, our Llama-3.1-8B model inherently makes this more challenging for us due to its limitations.** This level of challenge is important to understand and work with, since you are better-equipped to decompose harder challenges and leverage your tools to their fullest as you scale up. With that said, a multi-turn long-term-memory research agent implemented with Llama-8B is quite tedious at the moment, with many of the streamlines interfaces assuming a stronger model.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA
from langchain_nvidia_ai_endpoints._statics import MODEL_TABLE

# llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://llm_client:9000/v1")
llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-nano-8b-v1", base_url="http://llm_client:9000/v1")

## If you know structured output is supported and want less warnings:
MODEL_TABLE[llm.model].supports_structured_output = True 

In [7]:
##########################################################################
## This block is also provided to help with rendering some of the outputs.
## This notebook will likely generate a lot of output... so these are used periodically.

from html import escape
from IPython.display import display, Markdown, HTML, IFrame

def dd(summary, body, hidden=True):
    return f"<details{'' if hidden else ' open'}>\n\n<summary><b>{summary}</b></summary>\n\n{body}\n\n</details>"

def ul(items):
    items_str = "\n</li><li>\n".join([str(v) for v in items])
    return f"<ul><li>{items_str}</li></ul>"

def ol(items):
    items_str = "\n</li><li>\n".join([str(v) for v in items])
    return f"<ol><li>{items_str}</li></ol>"

def bq(body):
    return f"\n\n<blockquote>\n\n{body}\n\n</blockquote>"

def dd_render(i, q, msgs=[], ans=""):
    """Dropdown rendered. Useful mostly to help render answer traces"""
    transcript_lines = []
    for role, content in msgs["messages"]:
        role_lbl = "User" if role == "user" else "AI"
        transcript_lines.append(f"**{role_lbl}:**\n\n{content}\n")
    transcript_md = "\n---\n".join(transcript_lines)

    return dd(
        summary=f"{i}. {escape(q)}",
        body=(
            "**Final Answer**"
            + bq(
                str(
                    ans
                    + "\n\n"
                    + dd("Show transcript (messages)", bq(transcript_md))
                )
            )
        ),
    )

def html_preview(value, unhidden=[], preview_len=200, k=""):
    """A very specifically-implemented preview function. Grossly verfit for this notebook"""
    get_title = lambda v: " ".join(str(v).split()[:3]) + "..." if not isinstance(v, dict) else v.get("title", list(v.values())[0])
    if len(str(value)) < preview_len:
        return value
    if isinstance(value, dict):
        return ul([f"<b>{k}</b>: {v}" if len(str(v)) < preview_len else dd(k, html_preview(v, unhidden, preview_len, k), k not in unhidden) for k, v in value.items()])
    if isinstance(value, list):
        return ol([v if len(str(v)) < preview_len else dd(get_title(v), html_preview(v, unhidden, preview_len), k not in unhidden) for i, v in enumerate(value)])
    return value

display(HTML(dd("This is a dropdown list", "which hides more stuff like" + ul(range(5))) + "Pretty cool, right?"))

<hr><br>

## **Part 1:** Define The Planner

For the initial system, please make a minimal-viable "supervisor"-style element which tries to delegate tasks. This is a very vague definition, so technically a module that generates a list of tasks is technically viable. So let's start with that!

In [10]:
from pydantic import BaseModel, Field
from functools import partial
from typing import List

from course_utils import SCHEMA_HINT
from course_utils import get_schema_hint

##################################################################
## TODO: Create an LLM client with the sole intention of generating a plan.

## Planner LLM: keep it deterministic and schema-bound.
## NOTE: `llm` is already defined earlier in the notebook. We create a dedicated client for planning.
plan_llm = ChatNVIDIA(
    model=llm.model,
    base_url="http://llm_client:9000/v1",
    temperature=0.0,
)
class Plan(BaseModel):
    ## TODO: Define a variable of choice, including useful prompt engineering/restrictions
    steps: List[str] = Field(
        description=(
            "A short, high-signal plan as a list of steps. "
            "Each step must be a single sentence, start with a verb, and be <= 18 words. "
            "No bullets, no numbering, no newlines inside a step. "
            "Target 4-8 steps total."
        ),
        min_length=4,
        max_length=8,
    )

    ## TODO: Perform some more prompt engineering. Maybe consider including the schema_hint
schema_hint = get_schema_hint(Plan)

planning_prompt = (
    ChatPromptTemplate.from_messages([
        ("system", (
            "You are a master planner that decomposes a user request into an efficient, minimal plan.\n"
            "Rules:\n"
            "- Output MUST be valid JSON that matches the provided schema exactly.\n"
            "- Each step must be actionable, specific, and single-sentence.\n"
            "- Avoid fluff, explanations, and nested sub-steps.\n"
            "- Do not mention these rules or the schema in the output.\n\n"
            + SCHEMA_HINT
        )),
        ("placeholder", "{messages}"),
    ])
    .partial(schema_hint=schema_hint)
)

structured_plan_llm = plan_llm.with_structured_output(
    schema=Plan.model_json_schema(),
    strict=True,
)
## TODO: Construct the necessary components to create the chain
planning_chain = planning_prompt | structured_plan_llm

input_msgs = {"messages": [("user", "Can you help me learn more about LangGraph?")]}
for chunk in planning_chain.stream(input_msgs):
    print(chunk)

## Example testing script
step_buffer = []
for chunk in planning_chain.stream(input_msgs):
    if "steps" in chunk and chunk.get("steps"):
        if len(chunk.get("steps")) > len(step_buffer):
            if step_buffer:
                print(flush=True)
            step_buffer += [""]
            print(" - ", end="", flush=True)
        dlen = len(chunk.get("steps")[-1]) - len(step_buffer[-1])
        step_buffer[-1] = chunk.get("steps")[-1]
        print(step_buffer[-1][-dlen:], end="", flush=True)

{}
{'steps': ['']}
{'steps': ['Ask']}
{'steps': ['Ask me']}
{'steps': ['Ask me questions']}
{'steps': ['Ask me questions about']}
{'steps': ['Ask me questions about Lang']}
{'steps': ['Ask me questions about LangGraph']}
{'steps': ['Ask me questions about LangGraph to']}
{'steps': ['Ask me questions about LangGraph to deepen']}
{'steps': ['Ask me questions about LangGraph to deepen your']}
{'steps': ['Ask me questions about LangGraph to deepen your understanding']}
{'steps': ['Ask me questions about LangGraph to deepen your understanding.']}
{'steps': ['Ask me questions about LangGraph to deepen your understanding.', '']}
{'steps': ['Ask me questions about LangGraph to deepen your understanding.', 'Explore']}
{'steps': ['Ask me questions about LangGraph to deepen your understanding.', 'Explore online']}
{'steps': ['Ask me questions about LangGraph to deepen your understanding.', 'Explore online resources']}
{'steps': ['Ask me questions about LangGraph to deepen your understanding.', 'E

<br>

In an effort to help modularize this process for later, feel free to use this generator wrapper. This is effectively just the same process, but now yielding the results out to be processed by its caller:

In [11]:
def generate_thoughts(input_msgs, config=None):
    step_buffer = [""]
    for chunk in planning_chain.stream(input_msgs, config=config):
        if "steps" in chunk and chunk.get("steps"):
            if len(chunk.get("steps")) > len(step_buffer):
                yield step_buffer[-1]
                step_buffer += [""]
            dlen = len(chunk.get("steps")[-1]) - len(step_buffer[-1])
            step_buffer[-1] = chunk.get("steps")[-1]
    yield step_buffer[-1]
    # print("FINISHED", flush=True)

for thought in generate_thoughts(input_msgs):
    print(" -", thought)

 - Ask me questions about LangGraph to deepen your understanding.
 - Explore online resources and tutorials for LangGraph to learn at your own pace.
 - Participate in LangGraph-related discussions and forums to engage with the community.
 - Practice using LangGraph tools and features to apply your knowledge.
 - Set learning goals and track your progress to stay motivated.


<hr><br>

## **Task 2:** Define The Retrieval Sub-Process Mechanism

Now that we have a list of steps that we would like to consider, let's use them as a basis for searching the internet. Try implementing a searching mechanism of choice, and try to parallelize/batch this process if possible. 

- Feel free to implement `search_internet` and `retrieve_via_query` in a manner consistent with the warm-up (`DDGS` + `NVIDIARerank`), or maybe write up your own scheme that you think would be interesting. It may be interesting to implement a loop (agent-as-a-tool?) where you search, expand context, filter, and search again. Conceptually easy, but implementationally more involved.
- You may use the `tools` format if you want, but it will not be necessary. Do as you think is interesting.
- Our solutions did use `RunnableLambda(...).batch` at some point. Some solutions may also try to leverage `RunnableParallel`. Either-or may be useful, but are not required.

In [12]:
from ddgs import DDGS
import functools
from langchain_core.runnables import RunnableLambda
from course_utils import batch_process  # uses RunnableLambda(...).batch internally + progress bar

####################################################################
## TODO: Implement a "step researcher" mechanism of choice
# A small, cacheable search helper.
@functools.cache
def search_internet(final_query: str, max_results: int = 8):
    """
    Returns a list[dict] of DDGS results. Each result typically includes:
    { 'title': str, 'href': str, 'body': str, ... }
    """
    final_query = " ".join(final_query.split()).strip()
    if not final_query:
        return []

    # Keep it simple + robust across DDGS versions
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(final_query, max_results=max_results))
    except Exception as e:
        print(f"[search_internet] query failed: {final_query!r} -> {e}")
        results = []

    # Normalize / keep only fields we care about
    cleaned = []
    for r in results or []:
        if not isinstance(r, dict):
            continue
        cleaned.append(
            {
                "title": r.get("title", ""),
                "href": r.get("href", ""),
                "body": r.get("body", "") or r.get("snippet", "") or "",
                "query": final_query,
            }
        )
    return cleaned


def research_options(steps, *, max_results_per_step: int = 8):
    """
    For each planned step, produce a list of web snippets.
    Uses batching for parallelism.
    """
    # Clean / dedupe while keeping order
    steps = [s.strip() for s in steps if str(s).strip()]
    seen = set()
    steps = [s for s in steps if not (s in seen or seen.add(s))]

    def build_query(step: str) -> str:
        # Light prompt engineering: anchor to LangGraph so steps don't drift.
        return f"LangGraph {step}"

    queries = [build_query(step) for step in steps]

    # Batch the searches in parallel
    results_per_query = batch_process(
        lambda q: search_internet(q, max_results=max_results_per_step),
        queries,
        max_concurrency=12,
    )
    return results_per_query


search_retrievals = research_options(step_buffer)
HTML(html_preview({k: v for k, v in zip(step_buffer, search_retrievals)}, step_buffer))

  0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
from langchain_nvidia import NVIDIARerank
from langchain_core.documents import Document
import re

def _simple_token_overlap_score(query: str, text: str) -> float:
    q = set(re.findall(r"[a-z0-9]+", query.lower()))
    t = set(re.findall(r"[a-z0-9]+", text.lower()))
    if not q or not t:
        return 0.0
    # overlap normalized by query length (cheap + works surprisingly well on snippets)
    return len(q & t) / len(q)

def retrieve_via_query(context_rets, query: str, k: int = 5):
    query = str(query).strip()
    if not query:
        return []

    # Flatten: context_rets can be list[dict] or list[list[dict]]
    snippets = []
    if isinstance(context_rets, list) and context_rets and isinstance(context_rets[0], list):
        for group in context_rets:
            snippets.extend(group or [])
    else:
        snippets = context_rets or []

    # Build Documents
    docs = []
    for s in snippets:
        if not isinstance(s, dict):
            continue
        text = (s.get("body") or "").strip()
        if not text:
            continue
        docs.append(
            Document(
                page_content=text,
                metadata={
                    "title": s.get("title", ""),
                    "href": s.get("href", ""),
                    "source_query": s.get("query", ""),
                },
            )
        )

    if not docs:
        return []

    # --- Try NVIDIA rerank first ---
    try:
        reranker = NVIDIARerank(top_n=min(k, len(docs)))
        ranked_docs = reranker.compress_documents(docs, query=query)

        out = []
        for d in ranked_docs:
            out.append(
                {
                    "text": d.page_content,
                    "title": d.metadata.get("title", ""),
                    "href": d.metadata.get("href", ""),
                    "score": d.metadata.get("relevance_score", None),
                    "ranker": "nvidia_rerank",
                }
            )
        return out

    except Exception as e:
        # Typical here: 404 /v1/ranking not found
        print(f"[retrieve_via_query] Falling back to local ranking. Rerank failed: {e}")

    # --- Fallback: local overlap ranking ---
    scored = []
    for d in docs:
        score = _simple_token_overlap_score(query, d.page_content)
        scored.append((score, d))

    scored.sort(key=lambda x: x[0], reverse=True)
    top = scored[: min(k, len(scored))]

    out = []
    for score, d in top:
        out.append(
            {
                "text": d.page_content,
                "title": d.metadata.get("title", ""),
                "href": d.metadata.get("href", ""),
                "score": float(score),
                "ranker": "local_overlap",
            }
        )
    return out

filtered_results = [retrieve_via_query(search_retrievals, step, k=5) for step in step_buffer]
HTML(html_preview({k: v for k, v in zip(step_buffer, filtered_results)}))

/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:269: UserWarning: Default model is set as: 01-ai/yi-large. 
Set model using model parameter. 
To get available models use available_models property.
  warnings.warn(


[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 21:52:08 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f113768ef20>, 'url': 'http://llm_client:9000/v1/ranking', 'encoding': None, 'history': [], 'reason': 'Not Found', 'cookies': <RequestsCookieJar[]>, 'elapsed': datetime.timedelta(microseconds=40305), 'request': <PreparedRequest [POST]>, 'connection': <requests.adapters.HTTPAdapter object at 0x7f114dc18cd0>}
[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 21:52:08 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f113768f130>, 'url': 'http

<hr><br>

## **Part 3:** Creating The Research Pipeline

Now that we have some minimum-viable semblance of a supervisor/subordinate system, let's go ahead and orchestrate them in an interesting way. Feel free to come up with your own mechanism for "reasoning" about the question and "researching" the results. If you don't see a straightforward way to make it work, a default pool of prompts is offered below (possibly the ones we used).

In [26]:
## TODO: Define the structured prompt template. Doesn't have to be this!
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

agent_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a careful research assistant.\n"
        "You will be given a question plus intermediate research snippets (each snippet includes a URL).\n\n"
        "Hard rules (grader-facing):\n"
        "1) Use ONLY the provided snippets for factual claims.\n"
        "   - If a claim is not supported by the snippets, explicitly label it as a Prediction or Assumption.\n"
        "2) Your answer MUST contain at least three distinct reasoning steps with headings in this exact order:\n"
        "   Step 1 — Core concept(s)\n"
        "   Step 2 — Concrete example (include code or a worked mini-example)\n"
        "   Step 3 — How to verify / experiment / apply\n"
        "3) Minimum length: 220 words.\n"
        "4) Inline citations are REQUIRED for factual claims.\n"
        "   - Every factual sentence must end with a citation in parentheses containing the URL.\n"
        "   - Example: ... (https://example.com/page)\n"
        "   - The URL must be directly clickable (include the full https://... form).\n"
        "   - Do NOT use placeholder citations (e.g., 'Missing', 'Source X', 'example.com').\n"
        "5) Forward-looking content policy:\n"
        "   - You MUST include at least 3 specific, plausible future LangGraph features IF the user asks for future features.\n"
        "   - Label each as 'Prediction:' and do NOT cite it as fact unless a snippet explicitly supports it.\n"
        "   - After the predictions, add 2–4 sentences of critical analysis linking each prediction to observed trends in the snippets.\n"
        "6) Verification methodology requirements (Step 3):\n"
        "   - Include at least 3 concrete experiments and name at least 2 measurable metrics (e.g., latency, cost, success rate).\n"
        "   - Include a short CI/CD-style evaluation loop (what to run automatically, when, and what fails the build).\n"
        "7) Style requirements:\n"
        "   - Be conversational but scholarly: no filler like 'Sure, I'd be happy to help'.\n"
        "   - Each sentence must add information.\n"
        "8) If the question mentions LangGraph, explicitly connect each step back to LangGraph.\n"
        "9) If you include code, it must be valid Python and runnable as-is.\n"
        "   - No placeholders, no undefined variables, no pseudocode disguised as code.\n\n"
        "Output format:\n"
        "- Use the three Step headings exactly.\n"
        "- End with 'Sources:' and list 2–6 URLs that you actually cited inline (no extras).\n"
    ),
    ("placeholder", "{messages}"),
])

intermediate_prompt = (
    "Research step: {action}\n"
    "Top retrieved snippets (cite these URLs inline when you use them):\n"
    "{result}\n"
)

final_question = (
    "Now answer the original user question.\n"
    "Reminder: you MUST follow the 3-step structure and inline-cite URLs from the snippets.\n"
    "Question: {question}\n"
)

In [27]:
## TODO: Organize a systen  to reason about your question progressively.

def _format_sources_block(filtered_step_results, max_sources_per_step=3):
    """Make a compact snippet block the model can cite (only include items with valid URLs)."""
    lines = []
    used = 0

    for j, item in enumerate(filtered_step_results or [], start=1):
        title = (item.get("title") or "").strip()
        href = (item.get("href") or "").strip()
        text = (item.get("text") or "").strip().replace("\n", " ")

        # Skip sources without real URLs (prevents 'Missing' / uncitable snippets)
        if not href.startswith("http"):
            continue
        if not text:
            continue

        if len(text) > 450:
            text = text[:450] + "..."

        used += 1
        # Provide both a markdown link AND the raw URL for maximum "clickable URL" compatibility
        lines.append(
            f"Source {used}: {title}\n"
            f"URL: {href}\n"
            f"Link: [{title or href}]({href})\n"
            f"Snippet: {text}\n"
        )

        if used >= max_sources_per_step:
            break

    return "\n".join(lines).strip()

def run_research_agent(question: str, *, max_results_per_step: int = 8, top_k: int = 6):
    # 1) Plan
    plan_state = {"messages": [("user", question)]}
    steps = [s.strip() for s in generate_thoughts(plan_state) if str(s).strip()]

    # 2) Retrieve per step
    search_retrievals = research_options(steps, max_results_per_step=max_results_per_step)
    filtered_results = [retrieve_via_query(search_retrievals, step, k=top_k) for step in steps]

    # 3) Accumulate trace into messages
    trace_messages = [("user", question)]
    for action, step_results in zip(steps, filtered_results):
        result_block = _format_sources_block(step_results, max_sources_per_step=3)
        if not result_block:
            # Explicitly tell the model there is nothing to cite, so it must label claims as Prediction/Assumption
            result_block = "(no citable snippets retrieved for this step; if needed, label claims as Prediction/Assumption)"
        trace_messages.append(("user", intermediate_prompt.format(action=action, result=result_block)))

    # 4) Ask for final answer (with citations)
    trace_messages.append(("user", final_question.format(question=question)))

    chain = agent_prompt | llm | StrOutputParser()
    answer = chain.invoke({"messages": trace_messages})

    # 5) Keep a clean URL list (only valid URLs)
    sources = []
    for step_results in filtered_results:
        for r in step_results or []:
            href = (r.get("href") or "").strip()
            if href.startswith("http") and href not in sources:
                sources.append(href)

    return {
        "question": question,
        "steps": steps,
        "filtered_results": filtered_results,
        "trace_messages": trace_messages,
        "answer": answer,
        "sources": sources,
    }

# --- Single example run ---
question = "Can you help me learn more about LangGraph?"
out = run_research_agent(question)

print(out["answer"])

  0%|          | 0/5 [00:00<?, ?it/s]

[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 22:41:34 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f1136aae110>, 'url': 'http://llm_client:9000/v1/ranking', 'encoding': None, 'history': [], 'reason': 'Not Found', 'cookies': <RequestsCookieJar[]>, 'elapsed': datetime.timedelta(microseconds=38231), 'request': <PreparedRequest [POST]>, 'connection': <requests.adapters.HTTPAdapter object at 0x7f11376ae550>}
[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 22:41:34 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f1136aad1b0>, 'url': 'http

In [28]:
## ALTERNATIVE STARTING POINT, IF YOU WANT TO USE A FOR LOOP
QUESTION_COUNT = 8
question_list = [
    "Can you help me learn more about LangGraph? Specifically, can you tell me about Memory Management?",
    "Can you help me learn more about LangGraph? Specifically, can you tell me about Pregel?",
    "Can you help me learn more about LangGraph? Specifically, can you tell me about subgraphs?",
    "Can you help me learn more about LangGraph? Specifically, can you tell me about full-duplex communication?",
    "Can you help me learn more about LangGraph? Specifically, can you tell me about productionalization?",
    "Can you help me learn more about LangGraph? Specifically, can you tell me about how the visualization works behind?",
    "Can you help me learn more about LangGraph? Specifically, can you give me a simple example of parsing image into text?",
    "Can you help me learn more about LangGraph? Specifically, can you predict the future possible features?",
]
assert len(question_list) == QUESTION_COUNT

input_msgs_list = []
answer_list = []
trace_list = []

for i, q in enumerate(question_list, start=1):
    out = run_research_agent(q)
    input_msgs_list.append({"messages": out["trace_messages"]})
    answer_list.append(out["answer"])
    trace_list.append(out["trace_messages"])

    display(Markdown(dd_render(i, q, {"messages": out["trace_messages"]}, out["answer"])))

print("All Done! (This is also exactly what you'll want for Part 4.)")

  0%|          | 0/4 [00:00<?, ?it/s]

[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 22:41:49 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f1136ac7b80>, 'url': 'http://llm_client:9000/v1/ranking', 'encoding': None, 'history': [], 'reason': 'Not Found', 'cookies': <RequestsCookieJar[]>, 'elapsed': datetime.timedelta(microseconds=51901), 'request': <PreparedRequest [POST]>, 'connection': <requests.adapters.HTTPAdapter object at 0x7f114df15d50>}
[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 22:41:49 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f1136a9af80>, 'url': 'http

<details>

<summary><b>1. Can you help me learn more about LangGraph? Specifically, can you tell me about Memory Management?</b></summary>

**Final Answer**

<blockquote>

Step 1: Core Concept(s)

Memory management is a vital aspect of LangGraph, as it ensures the efficient allocation and deallocation of memory resources, which is crucial for building intelligent and context-aware applications. LangGraph, like other memory management systems, employs various strategies to manage memory, ensuring that memory is used optimally without causing performance issues. (Source 3: https://github.com/FareedKhan-dev/langgraph-long-memory)

Step 2: Concrete Example

Let's consider the concept of memory management in LangGraph more closely. One of the key features of LangGraph is its Store feature, which manages how memory is saved, retrieved, and updated, depending on where you are running your project. For example, if you are running LangGraph in-memory, the Store feature helps ensure that memory is allocated and deallocated efficiently. (Source 1: https://github.com/FareedKhan-dev/langgraph-long-memory)

Source 2: No snippet provided for this step, but the main idea is to illustrate how the Store feature works in different environments.

Step 3: How to Verify / Experiment / Apply

To verify or experiment with memory management in LangGraph, you can explore its official documentation and guides, such as the Comprehensive Guide: Long-Term Agentic Memory With LangGraph (Source 2: https://medium.com/@anil.jain.baba/long-term-agentic-memory-with-langgraph-824050b09852). These resources will guide you through the process of setting up memory management, allocating and deallocating memory, and ensuring its efficiency.

To apply this knowledge, you can look for tutorials and examples online, such as the Reddit thread titled: "How to implement memory saving in Langgraph agents" (Source 3: https://www.reddit.com/r/LangChain/comments/1kwlhed/how_to_implement_memory_saving_in_langgraph_agents/). These tutorials will practice your memory management skills.

Sources:

* https://github.com/FareedKhan-dev/langgraph-long-memory
* https://medium.com/@anil.jain.baba/long-term-agentic-memory-with-langgraph-824050b09852
* https://www.reddit.com/r/LangChain/comments/1kwlhed/how_to_implement_memory_saving_in_langgraph_agents/

<details>

<summary><b>Show transcript (messages)</b></summary>



<blockquote>

**User:**

Can you help me learn more about LangGraph? Specifically, can you tell me about Memory Management?

---
**User:**

Research step: Explain the basics of memory management in LangGraph.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Understanding Memory Management in LangGraph: A Hands-On Guide | by Suraj Pandey | Medium
URL: https://medium.com/@suraj.pandey199227/understanding-memory-management-in-langgraph-a-hands-on-guide-5dd207a58bbf
Link: [Understanding Memory Management in LangGraph: A Hands-On Guide | by Suraj Pandey | Medium](https://medium.com/@suraj.pandey199227/understanding-memory-management-in-langgraph-a-hands-on-guide-5dd207a58bbf)
Snippet: August 29, 2025 - In this guide, we’ll dive deep into LangGraph’s memory management system, exploring different types of memory, practical implementation strategies, and real-world use cases that will help you build more intelligent and context-aware applications.

Source 2: Comprehensive Guide: Long-Term Agentic Memory With LangGraph | by Anil Jain | AI / ML Architect | Data Architect | Medium
URL: https://medium.com/@anil.jain.baba/long-term-agentic-memory-with-langgraph-824050b09852
Link: [Comprehensive Guide: Long-Term Agentic Memory With LangGraph | by Anil Jain | AI / ML Architect | Data Architect | Medium](https://medium.com/@anil.jain.baba/long-term-agentic-memory-with-langgraph-824050b09852)
Snippet: April 19, 2025 - As explained in the LangGraph documentation, “ Short-term memory, or thread-scoped memory, can be recalled at any time from within a single conversational thread with a user . LangGraph manages short-term memory as a part of your agent’s state” ...

Source 3: GitHub - FareedKhan-dev/langgraph-long-memory: A detail Implementation of handling long-term memory in Agentic AI
URL: https://github.com/FareedKhan-dev/langgraph-long-memory
Link: [GitHub - FareedKhan-dev/langgraph-long-memory: A detail Implementation of handling long-term memory in Agentic AI](https://github.com/FareedKhan-dev/langgraph-long-memory)
Snippet: LangGraph is the most favorable ... and one of the most common features is the Store feature in LangGraph, which manages how memory is saved, retrieved, and updated, depending on where you are running your project ...


---
**User:**

Research step: Discuss how LangGraph handles memory allocation and deallocation.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: GitHub - FareedKhan-dev/langgraph-long-memory: A detail Implementation of handling long-term memory in Agentic AI
URL: https://github.com/FareedKhan-dev/langgraph-long-memory
Link: [GitHub - FareedKhan-dev/langgraph-long-memory: A detail Implementation of handling long-term memory in Agentic AI](https://github.com/FareedKhan-dev/langgraph-long-memory)
Snippet: LangGraph is the most favorable ... and one of the most common features is the Store feature in LangGraph, which manages how memory is saved, retrieved, and updated, depending on where you are running your project ...

Source 2: GitHub - langchain-ai/langgraph-memory
URL: https://github.com/langchain-ai/langgraph-memory
Link: [GitHub - langchain-ai/langgraph-memory](https://github.com/langchain-ai/langgraph-memory)
Snippet: The memory graph handles thread process deduplication and supports continuous updates to a single "memory schema" as well as "event-based" memories that can be queried semantically. ├── langgraph.json # LangGraph Cloud Configuration ├── memory_service │ ├── __init__.py │ └── graph.py # Define the memory service ├── poetry.lock ├── pyproject.toml # Project dependencies └── tests # Add testing + evaluation logic └── evals └── test_memories.py

Source 3: Understanding Memory Management in LangGraph: A Hands-On Guide | by Suraj Pandey | Medium
URL: https://medium.com/@suraj.pandey199227/understanding-memory-management-in-langgraph-a-hands-on-guide-5dd207a58bbf
Link: [Understanding Memory Management in LangGraph: A Hands-On Guide | by Suraj Pandey | Medium](https://medium.com/@suraj.pandey199227/understanding-memory-management-in-langgraph-a-hands-on-guide-5dd207a58bbf)
Snippet: August 29, 2025 - In this guide, we’ll dive deep into LangGraph’s memory management system, exploring different types of memory, practical implementation strategies, and real-world use cases that will help you build more intelligent and context-aware applications.


---
**User:**

Research step: Provide an example of memory management in LangGraph code.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Understanding Memory Management in LangGraph: A Hands-On Guide | by Suraj Pandey | Medium
URL: https://medium.com/@suraj.pandey199227/understanding-memory-management-in-langgraph-a-hands-on-guide-5dd207a58bbf
Link: [Understanding Memory Management in LangGraph: A Hands-On Guide | by Suraj Pandey | Medium](https://medium.com/@suraj.pandey199227/understanding-memory-management-in-langgraph-a-hands-on-guide-5dd207a58bbf)
Snippet: August 29, 2025 - In this guide, we’ll dive deep into LangGraph’s memory management system, exploring different types of memory, practical implementation strategies, and real-world use cases that will help you build more intelligent and context-aware applications.

Source 2: Understanding Memory Management in LangGraph: A Hands-On Guide | by Suraj Pandey | Medium
URL: https://medium.com/@suraj.pandey199227/understanding-memory-management-in-langgraph-a-hands-on-guide-5dd207a58bbf
Link: [Understanding Memory Management in LangGraph: A Hands-On Guide | by Suraj Pandey | Medium](https://medium.com/@suraj.pandey199227/understanding-memory-management-in-langgraph-a-hands-on-guide-5dd207a58bbf)
Snippet: August 29, 2025 - In this guide, we’ll dive deep into LangGraph’s memory management system, exploring different types of memory, practical implementation strategies, and real-world use cases that will help you build more intelligent and context-aware applications.

Source 3: How to implement memory saving in Langgraph agents - Reddit
URL: https://www.reddit.com/r/LangChain/comments/1kwlhed/how_to_implement_memory_saving_in_langgraph_agents/
Link: [How to implement memory saving in Langgraph agents - Reddit](https://www.reddit.com/r/LangChain/comments/1kwlhed/how_to_implement_memory_saving_in_langgraph_agents/)
Snippet: May 27, 2025 · In the tutorial the show how the graph.compile() method can receive a memorysaver parameter while they also show how we can bind memory saving tools to the llm. Anyone has a langchain example of how to use memory? - Reddit How to implement memory in separate db in langgraph ? : r/LangChain More results from www.reddit.com


---
**User:**

Research step: Describe the garbage collection process in LangGraph.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Comprehensive Guide: Long-Term Agentic Memory With LangGraph | by Anil Jain | AI / ML Architect | Data Architect | Medium
URL: https://medium.com/@anil.jain.baba/long-term-agentic-memory-with-langgraph-824050b09852
Link: [Comprehensive Guide: Long-Term Agentic Memory With LangGraph | by Anil Jain | AI / ML Architect | Data Architect | Medium](https://medium.com/@anil.jain.baba/long-term-agentic-memory-with-langgraph-824050b09852)
Snippet: April 19, 2025 - As explained in the LangGraph documentation, “ Short-term memory, or thread-scoped memory, can be recalled at any time from within a single conversational thread with a user . LangGraph manages short-term memory as a part of your agent’s state” ...

Source 2: Customizing Memory in LangGraph Agents for Better Conversations | Focused
URL: https://focused.io/lab/customizing-memory-in-langgraph-agents-for-better-conversations
Link: [Customizing Memory in LangGraph Agents for Better Conversations | Focused](https://focused.io/lab/customizing-memory-in-langgraph-agents-for-better-conversations)
Snippet: 1 week ago - LangGraph uses checkpointers (e.g., InMemorySaver for in-memory, SqliteSaver for persistent storage) to manage state across conversations . For more on the migration, refer to the official migration guide.

Source 3: GitHub - FareedKhan-dev/langgraph-long-memory: A detail Implementation of handling long-term memory in Agentic AI
URL: https://github.com/FareedKhan-dev/langgraph-long-memory
Link: [GitHub - FareedKhan-dev/langgraph-long-memory: A detail Implementation of handling long-term memory in Agentic AI](https://github.com/FareedKhan-dev/langgraph-long-memory)
Snippet: LangGraph is the most favorable ... and one of the most common features is the Store feature in LangGraph, which manages how memory is saved, retrieved, and updated, depending on where you are running your project ...


---
**User:**

Now answer the original user question.
Reminder: you MUST follow the 3-step structure and inline-cite URLs from the snippets.
Question: Can you help me learn more about LangGraph? Specifically, can you tell me about Memory Management?



</blockquote>

</details>

</blockquote>

</details>

  0%|          | 0/5 [00:00<?, ?it/s]

[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 22:41:58 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f1136ac6c50>, 'url': 'http://llm_client:9000/v1/ranking', 'encoding': None, 'history': [], 'reason': 'Not Found', 'cookies': <RequestsCookieJar[]>, 'elapsed': datetime.timedelta(microseconds=39473), 'request': <PreparedRequest [POST]>, 'connection': <requests.adapters.HTTPAdapter object at 0x7f11500bcc90>}
[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 22:41:58 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f1136ac6710>, 'url': 'http

...

<details>

<summary><b>7. Can you help me learn more about LangGraph? Specifically, can you give me a simple example of parsing image into text?</b></summary>

**Final Answer**

<blockquote>

**Step 1: Core Concept**

To parse an image into text, we can utilize Tesseract, which is built on top of Language Detection for Python (LangDetect-PY). Tesseract is particularly useful for images to convert them into readable text. Here's the first step to understand Tesseract as a text recognition library in the context of LangGraph:

* <br> **Concept:** Tesseract (or pytesseract) is a Python library that uses OpenCV to open an image, then applies the Tesseract-OCR (Optical Character Recognition) engine to convert the image text into plain text. This is used for text recognition in various forms, including images.

**Source 1:** Unlocking Text from Images: An Introduction to Tesseract for [langchain.com](https://typevar.dev/articles/tesseract-ocr/tesseract)

**Snippet:** Sep 11, 2025 · Here's a simple Python example using Pytesseract to extract text from an image. This code demonstrates the core functionality you'll use in your projects.

URL: https://typevar.dev/articles/tesseract-ocr/tesseract

**Step 2: Concrete Example**

Here's a simple concrete example using Pytesseract and Pillow:

```python
from PIL import Image
import pytesseract
    
def parse_image_to_text(image_path):
    # Load the image
    img = Image.open(image_path)
    
    # Convert image to bytes
    img_bytes = img.read()
    
    # Extract the text using Tesseract
    text = pytesseract.image_to_string(img_bytes)
    
    # Return the extracted text
    return text.strip()

# Example usage
image_path = 'path/to/your/image.jpg'
print(parse_image_to_text(image_path))
```

**Source 2:** How to Create an Image to Text Converter Python | Step-by-Step Guide [taglineinfotech.com](https://taglineinfotech.com/blog	image-to-text-converter-python-tutorial/)

**Snippet:** This will create a new file called extracted text.txt and save all the extracted text inside it. You've successfully created your own image-to-text converter. Now all you need to do is to change the image path, run the same commands, and start extracting the text.

URL: https://taglineinfotech.com/blog/image-to-text-converter-python-tutorial/

**Step 3: How to Verify/Applying**

To verify or apply the extracted text from an image, you can simply follow a similar process as in the example above or make use of the output text from the function:

* <br> **Task:** Replace the `image_path` in the `parse_image_to_text` function call with the path to the image file you wish to process.
* <br> **Verification:** The function will return the extracted text, which can be printed or saved to a file.

**Sources Used:**
* [Unlocking Text from Images: An Introduction to Tesseract for](https://typevar.dev/articles/tesseract-ocr/tesseract)
* [How to Create an Image to Text Converter Python | Step-by-Step Guide](https://taglineinfotech.com/blog/image-to-text-converter-python-tutorial/)
```

**Sources:** [**Literature Retrieved from Snippets, Inline Cited Appropriately]**

**Final Instructions:** For more such answers, follow the same structure and inline-cite URLs from the snippets. Leave questions or instructions in the box if you need further assistance.

<details>

<summary><b>Show transcript (messages)</b></summary>



<blockquote>

**User:**

Can you help me learn more about LangGraph? Specifically, can you give me a simple example of parsing image into text?

---
**User:**

Research step: Parse image using a library like Pillow to load image data.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Tutorial - Pillow (PIL Fork) 12.1.1 documentation
URL: https://pillow.readthedocs.io/en/stable/handbook/tutorial.html
Link: [Tutorial - Pillow (PIL Fork) 12.1.1 documentation](https://pillow.readthedocs.io/en/stable/handbook/tutorial.html)
Snippet: Note that the library rewinds the file (using seek(0)) before reading the image header. In addition, seek will also be used when the image data is read (by the load method). If the image file is embedded in a larger file, such as a tar file, you can use the ContainerIO or TarIO modules to access it. from PIL import Image from urllib.request import urlopen url = "https://python-pillow.github.io/assets/images/pillow-logo.png" img = Image.open(urlop...

Source 2: Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium
URL: https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39
Link: [Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium](https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39)
Snippet: May 19, 2025 - The inner loop extracts the most recent assistant-generated message from the streamed event, pretty-prints it, parses it using a parser object, and then sends the parsed response back to the client over the WebSocket. So that’s it! We’ve built a simple visual data extraction agent using LangGraph and LangChain. ... The AI Agent intelligently extracts structured product data from the uploaded image menu, summarizes details via conve...

Source 3: Unlocking Text from Images: An Introduction to Tesseract for ...
URL: https://typevar.dev/articles/tesseract-ocr/tesseract
Link: [Unlocking Text from Images: An Introduction to Tesseract for ...](https://typevar.dev/articles/tesseract-ocr/tesseract)
Snippet: Sep 11, 2025 · Here's a simple Python example using Pytesseract to extract text from an image . This code demonstrates the core functionality you'll use in your projects. Pytesseract often works best with the Python Imaging Library (PIL) for opening image files.


---
**User:**

Research step: Use a text recognition library like Tesseract to convert image text into text format.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Django X Tesseract in action. | Python in Plain English
URL: https://python.plainenglish.io/an-introduction-to-ocr-technology-with-python-django-and-tesseract-595843ad73b4?source=user_profile---------3----------------------------
Link: [Django X Tesseract in action. | Python in Plain English](https://python.plainenglish.io/an-introduction-to-ocr-technology-with-python-django-and-tesseract-595843ad73b4?source=user_profile---------3----------------------------)
Snippet: We’ll convert an image into a text , and by the end, you’ll be amazed by how easy that was. First and foremost, we need to make sure we have the Tesseract OCR engine installed in our system.From Zero to Hero: Building an Intelligent Chatbot with LangGraph in No Time.

Source 2: Unlocking Text from Images: An Introduction to Tesseract for ...
URL: https://typevar.dev/articles/tesseract-ocr/tesseract
Link: [Unlocking Text from Images: An Introduction to Tesseract for ...](https://typevar.dev/articles/tesseract-ocr/tesseract)
Snippet: Sep 11, 2025 · Here's a simple Python example using Pytesseract to extract text from an image . This code demonstrates the core functionality you'll use in your projects. Pytesseract often works best with the Python Imaging Library (PIL) for opening image files.

Source 3: Tesseract OCR Software Tutorial - New York University
URL: https://guides.nyu.edu/tesseract/converting
Link: [Tesseract OCR Software Tutorial - New York University](https://guides.nyu.edu/tesseract/converting)
Snippet: Oct 20, 2025 · A step-by-step guide for users to learn how to use Tesseract open-source software for performing optical character recognition (OCR) on a text corpus.


---
**User:**

Research step: Save the extracted text to a file or display it.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: How to Create an Image to Text Converter Python | Step-by-Step Guide
URL: https://taglineinfotech.com/blog/image-to-text-converter-python-tutorial/
Link: [How to Create an Image to Text Converter Python | Step-by-Step Guide](https://taglineinfotech.com/blog/image-to-text-converter-python-tutorial/)
Snippet: This will create a new file called extracted _ text .txt and save all the extracted text inside it . You’ve successfully created your own image-to- text converter. Now all you need to do is to change the image path, run the same commands, and start extracting the text .

Source 2: LangGraph : Build Stateful AI Agents in Python – Real Python
URL: https://realpython.com/langgraph-python/
Link: [LangGraph : Build Stateful AI Agents in Python – Real Python](https://realpython.com/langgraph-python/)
Snippet: Also, because exclude is True and repr is False in the definition of date_of_notice_str , you won’t see date_of_notice_str when you serialize or display NoticeEmailExtract .You then save the image to a file called notice_ extraction _ graph .png . Here’s what the image should look like

Source 3: Image to Text Converter - OCR to Extract Text from Image
URL: https://www.cardscanner.co/image-to-text
Link: [Image to Text Converter - OCR to Extract Text from Image](https://www.cardscanner.co/image-to-text)
Snippet: Save Converted Text : Copy the extracted text or click “Download” and save it into the desired text format. These are the reasons that make this OCR tool best to scan and extract text from images or PDFs: Batch Files . OCR for multiple files at once.


---
**User:**

Research step: title
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Background job for generating titles of chats #152 - GitHub
URL: https://github.com/langchain-ai/open-canvas/issues/152
Link: [Background job for generating titles of chats #152 - GitHub](https://github.com/langchain-ai/open-canvas/issues/152)
Snippet: Oct 28, 2024 · Once generated, it should then call the LangGraph SDK to update the metadata on the thread with a new field thread_ title . Update the client side code which gets the title for the thread to first look in the thread metadata for this field, and if it doesn't exist, fallback to the current logic.

Source 2: langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation
URL: https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html
Link: [langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation](https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html)
Snippet: Yields: Document: A document containing the parsed content and metadata. """ try: from PIL import Image as Img except ImportError: raise ImportError( "`Pillow` package not found, please install it with " "`pip install Pillow`" ) with blob.as_bytes_io() as buf: if blob.mimetype == "application/x-npy": array = numpy.load(buf) if array.ndim == 3 and array.shape[2] == 1: # Grayscale image img = Img.fromarray(numpy.squeeze(array, axis=2), mode="L") el...

Source 3: Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium
URL: https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39
Link: [Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium](https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39)
Snippet: May 19, 2025 - The inner loop extracts the most recent assistant-generated message from the streamed event, pretty-prints it, parses it using a parser object, and then sends the parsed response back to the client over the WebSocket. So that’s it! We’ve built a simple visual data extraction agent using LangGraph and LangChain. ... The AI Agent intelligently extracts structured product data from the uploaded image menu, summarizes details via conve...


---
**User:**

Research step: type
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Three LangGraph Agent Patterns That Replaced Hundreds of Lines of Glue ...
URL: https://dev.to/irubtsov/three-langgraph-agent-patterns-that-replaced-hundreds-of-lines-of-glue-code-3a21
Link: [Three LangGraph Agent Patterns That Replaced Hundreds of Lines of Glue ...](https://dev.to/irubtsov/three-langgraph-agent-patterns-that-replaced-hundreds-of-lines-of-glue-code-3a21)
Snippet: When to use it: Any time your workflow needs to branch — different processing paths based on analysis results, document type classification, or risk level assessment. Most routing in LangGraph tutorials is done with conditional edges and deterministic functions. That's fine for simple cases.

Source 2: langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation
URL: https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html
Link: [langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation](https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html)
Snippet: Yields: Document: A document containing the parsed content and metadata. """ try: from PIL import Image as Img except ImportError: raise ImportError( "`Pillow` package not found, please install it with " "`pip install Pillow`" ) with blob.as_bytes_io() as buf: if blob.mimetype == "application/x-npy": array = numpy.load(buf) if array.ndim == 3 and array.shape[2] == 1: # Grayscale image img = Img.fromarray(numpy.squeeze(array, axis=2), mode="L") el...

Source 3: Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium
URL: https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39
Link: [Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium](https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39)
Snippet: May 19, 2025 - The inner loop extracts the most recent assistant-generated message from the streamed event, pretty-prints it, parses it using a parser object, and then sends the parsed response back to the client over the WebSocket. So that’s it! We’ve built a simple visual data extraction agent using LangGraph and LangChain. ... The AI Agent intelligently extracts structured product data from the uploaded image menu, summarizes details via conve...


---
**User:**

Research step: items
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Storage (LangGraph) | LangChain Reference
URL: https://reference.langchain.com/python/langgraph/store/
Link: [Storage (LangGraph) | LangChain Reference](https://reference.langchain.com/python/langgraph/store/)
Snippet: PutOp Bases: NamedTuple Operation to store, update, or delete an item in the store. This class represents a single operation to modify the store's contents, whether adding new items , updating existing ones, or removing them.

Source 2: langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation
URL: https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html
Link: [langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation](https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html)
Snippet: Yields: Document: A document containing the parsed content and metadata. """ try: from PIL import Image as Img except ImportError: raise ImportError( "`Pillow` package not found, please install it with " "`pip install Pillow`" ) with blob.as_bytes_io() as buf: if blob.mimetype == "application/x-npy": array = numpy.load(buf) if array.ndim == 3 and array.shape[2] == 1: # Grayscale image img = Img.fromarray(numpy.squeeze(array, axis=2), mode="L") el...

Source 3: Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium
URL: https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39
Link: [Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium](https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39)
Snippet: May 19, 2025 - The inner loop extracts the most recent assistant-generated message from the streamed event, pretty-prints it, parses it using a parser object, and then sends the parsed response back to the client over the WebSocket. So that’s it! We’ve built a simple visual data extraction agent using LangGraph and LangChain. ... The AI Agent intelligently extracts structured product data from the uploaded image menu, summarizes details via conve...


---
**User:**

Research step: maxItems
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation
URL: https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html
Link: [langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation](https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html)
Snippet: Yields: Document: A document containing the parsed content and metadata. """ try: from PIL import Image as Img except ImportError: raise ImportError( "`Pillow` package not found, please install it with " "`pip install Pillow`" ) with blob.as_bytes_io() as buf: if blob.mimetype == "application/x-npy": array = numpy.load(buf) if array.ndim == 3 and array.shape[2] == 1: # Grayscale image img = Img.fromarray(numpy.squeeze(array, axis=2), mode="L") el...

Source 2: Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium
URL: https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39
Link: [Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium](https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39)
Snippet: May 19, 2025 - The inner loop extracts the most recent assistant-generated message from the streamed event, pretty-prints it, parses it using a parser object, and then sends the parsed response back to the client over the WebSocket. So that’s it! We’ve built a simple visual data extraction agent using LangGraph and LangChain. ... The AI Agent intelligently extracts structured product data from the uploaded image menu, summarizes details via conve...

Source 3: Image Processing using Pillow by Sunny Solanki
URL: https://coderzcolumn.com/tutorials/python/pillow
Link: [Image Processing using Pillow by Sunny Solanki](https://coderzcolumn.com/tutorials/python/pillow)
Snippet: Python Imaging Library is used for loading images in memory and doing various transformations on the image . It supports loading images of various types like PNG, JPEG, etc. It can be used to do various transformations on image and then store ...


---
**User:**

Research step: minItems
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation
URL: https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html
Link: [langchain_community.document_loaders.parsers.images — 🦜🔗 LangChain documentation](https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/parsers/images.html)
Snippet: Yields: Document: A document containing the parsed content and metadata. """ try: from PIL import Image as Img except ImportError: raise ImportError( "`Pillow` package not found, please install it with " "`pip install Pillow`" ) with blob.as_bytes_io() as buf: if blob.mimetype == "application/x-npy": array = numpy.load(buf) if array.ndim == 3 and array.shape[2] == 1: # Grayscale image img = Img.fromarray(numpy.squeeze(array, axis=2), mode="L") el...

Source 2: Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium
URL: https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39
Link: [Learning LangGraph: Building a visual data extraction agent | by Salraihan | Medium](https://medium.com/@salraihan444/learning-langgraph-building-a-visual-data-extraction-agent-7ce18b7bae39)
Snippet: May 19, 2025 - The inner loop extracts the most recent assistant-generated message from the streamed event, pretty-prints it, parses it using a parser object, and then sends the parsed response back to the client over the WebSocket. So that’s it! We’ve built a simple visual data extraction agent using LangGraph and LangChain. ... The AI Agent intelligently extracts structured product data from the uploaded image menu, summarizes details via conve...

Source 3: Image Processing using Pillow by Sunny Solanki
URL: https://coderzcolumn.com/tutorials/python/pillow
Link: [Image Processing using Pillow by Sunny Solanki](https://coderzcolumn.com/tutorials/python/pillow)
Snippet: Python Imaging Library is used for loading images in memory and doing various transformations on the image . It supports loading images of various types like PNG, JPEG, etc. It can be used to do various transformations on image and then store ...


---
**User:**

Now answer the original user question.
Reminder: you MUST follow the 3-step structure and inline-cite URLs from the snippets.
Question: Can you help me learn more about LangGraph? Specifically, can you give me a simple example of parsing image into text?



</blockquote>

</details>

</blockquote>

</details>

  0%|          | 0/4 [00:00<?, ?it/s]

[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 22:43:06 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f1136ac7280>, 'url': 'http://llm_client:9000/v1/ranking', 'encoding': None, 'history': [], 'reason': 'Not Found', 'cookies': <RequestsCookieJar[]>, 'elapsed': datetime.timedelta(microseconds=34531), 'request': <PreparedRequest [POST]>, 'connection': <requests.adapters.HTTPAdapter object at 0x7f11501e9ad0>}
[retrieve_via_query] Falling back to local ranking. Rerank failed: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'date': 'Fri, 13 Feb 2026 22:43:06 GMT', 'server': 'uvicorn', 'content-length': '19'}, 'raw': <urllib3.response.HTTPResponse object at 0x7f1136ac7610>, 'url': 'http

<details>

<summary><b>8. Can you help me learn more about LangGraph? Specifically, can you predict the future possible features?</b></summary>

**Final Answer**

<blockquote>

Step 1: Core Concept

LangGraph is a flexible AI framework designed for building and orchestrating reliable and stateful AI agents. Its primary goal is to support diverse control flows, such as single-agent, multi-agent, hierarchical, and sequential, while handling complex and realistic scenarios. To proceed, I will analyze user feedback, trends, and recent advancements in the field to predict potential future features.

Source 1: LangGraph (LangGraph UserGroup) - Hugging Face ( Inline-Citation: https://www.langchain.com/langgraph )
Snippet: Mar 14, 2025 · We encourage you to join the LangGraph User Group to connect with fellow developers, ask questions, share your projects, and contribute to the LangGraph ecosystem.

Step 2: Concrete Example

One potential feature is the integration of more advanced natural language processing (NLP) techniques, such as transformer models like BERT or GPT-3, into LangGraph. These models offer improved language understanding and generation capabilities, which could be leveraged to create more complex and context-aware agents. For instance, a development team could build a LangGraph agent that interprets human instructions in various languages and supplies tailored responses, demonstrating its added NLP capabilities. This feature could be showcased in a project like badge-grade badminton tournaments, where AI-powered systems assist coaches with game recommendations and strategy suggestions.  (Source 2: Real-World Applications and Case Studies with LangGraph: From ... - Medium ( Use inline citation ) )

URL: https://medium.com/@garima_yadav/real-world-applications-and-case-studies-with-langgraph-from-the-theory-to-practice-7a6ffd2e8e1b
Link: [Real-World Applications and Case Studies with LangGraph: From ... - Medium](https://medium.com/@garima_yadav/real-world-applications-and-case-studies-with-langgraph-from_the-theory-to-practice-7a6bfdca92f832)

Step 3: Verification and Application

To verify the impact of new features, developers could monitor agent performance metrics, such as latency, cost, and success rate, to ensure a reliable production-ready system. To apply new features, one could transfer a LangGraph agent to a new development environment and introduce the new NLP capabilities. Here, a team could use LangGraph's tools to evaluate how well the AI interprets and responds to an large number of diverse human inputs, replicating a real-world scenario. This step confirms that the new feature meets the community's needs and is beneficial for the LangGraph ecosystem.  (Source 1: example_langgraph_agents.ipynb - Colab ( Use inline citation ) )

URL: https://colab.research.google.com/github/langfuse/langfuse-docs/blob/main/cookbook/example_langgraph_agents.ipynb
Link: [example_langgraph_agents.ipynb - Colab](https://colab.research.google.com/github/langfuse/langfuse-docs/blob/main/cookbook/example_langgraph_agents.ipynb)

Sources: 

- (https://www.langchain.com/langgraph/)
- (https://medium.com/@garima_yadav/real-world-applications-and-case-studies-with-langgraph-from_the-theory-to-practice-7a6fd3ca9fff860)
- (https://colab.research.google.com/github/langfuse/langfuse-docs/blob/main/cookbook/example_langgraph_agents.ipynb)

Additionally, consider these future LangGraph features:

* Integration of new machine learning techniques, such as reinforcement learning and learned appearance models.
* Enhanced support for cross-ended collaboration and interface designs.
* Open-source contributions and community-driven improvements. 

These predictions are informed by the community's feedback, latest advancements, and the need for more comprehensive and versatile AI systems.  (Source 3: LangGraph for Healthcare: A Comprehensive Technical Guide ( Use inline citation ) )

URL: https://levelup.gitconnected.com/langgraph-for-healthcare-a-comprehensive-technical-guide-e6038b06c108
Link: [LangGraph for Healthcare: A Comprehensive Technical Guide](https://levelup.gitconnected.com/langgraph-for-healthcare-a-comprehensive-technical-guide-e6038b06c108)

Note: 

* The analysis of future features is based on publicly available information and community feedback.
* No specific claim of prophecy is made; future features are speculative.
* The elements listed in "Additionally" are open to community scrutiny and experimentation, but their development and deployment will be cross-checked for compatibility with the LangGraph framework.

<details>

<summary><b>Show transcript (messages)</b></summary>



<blockquote>

**User:**

Can you help me learn more about LangGraph? Specifically, can you predict the future possible features?

---
**User:**

Research step: Explore existing resources on LangGraph to gain a solid foundation in its current features and capabilities.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Guide to Building AI Agents with LangGraph
URL: https://www.ema.co/additional-blogs/addition-blogs/building-ai-agents-langgraph
Link: [Guide to Building AI Agents with LangGraph](https://www.ema.co/additional-blogs/addition-blogs/building-ai-agents-langgraph)
Snippet: Jul 14, 2025 · In this blog, you'll get a clear, step-by-step guide to building AI agents with LangGraph. We'll cover what LangGraph is, its top features, the ... Missing: resources gain solid

Source 2: LangGraph overview
URL: https://docs.langchain.com/oss/python/langgraph/overview
Link: [LangGraph overview](https://docs.langchain.com/oss/python/langgraph/overview)
Snippet: LangGraph is focused on the underlying capabilities important for agent orchestration: durable execution, streaming, human-in-the-loop, and more. . Install. Missing: solid foundation

Source 3: LangGraph AI Framework 2025: Complete Architecture Guide + Multi-Agent ...
URL: https://latenode.com/blog/ai-frameworks-technical-infrastructure/langgraph-multi-agent-orchestration/langgraph-ai-framework-2025-complete-architecture-guide-multi-agent-orchestration-analysis
Link: [LangGraph AI Framework 2025: Complete Architecture Guide + Multi-Agent ...](https://latenode.com/blog/ai-frameworks-technical-infrastructure/langgraph-multi-agent-orchestration/langgraph-ai-framework-2025-complete-architecture-guide-multi-agent-orchestration-analysis)
Snippet: LangGraph's standout feature is its ability to maintain shared, persistent states across workflows, allowing dynamic adjustments based on runtime conditions. For example, in a document review system, agents can analyze text, flag issues, and pause for human feedback while retaining all prior context.


---
**User:**

Research step: Analyze user feedback and trends to predict potential future features that would enhance the platform's usability and appeal.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: LangGraph: Agent Orchestration Framework for Reliable AI Agents - LangChain
URL: https://www.langchain.com/langgraph
Link: [LangGraph: Agent Orchestration Framework for Reliable AI Agents - LangChain](https://www.langchain.com/langgraph)
Snippet: LangGraph's flexible framework supports diverse control flows - single agent, multi-agent, hierarchical, sequential - and robustly handles realistic, complex scenarios. Ensure reliability with easy- to -add moderation and quality loops that prevent agents from veering off course. Use LangGraph Platform to templatize your cognitive architecture so that tools, prompts, and models are easily ...

Source 2: LangGraph AI Framework 2025: Complete Architecture Guide + Multi-Agent ...
URL: https://latenode.com/blog/ai-frameworks-technical-infrastructure/langgraph-multi-agent-orchestration/langgraph-ai-framework-2025-complete-architecture-guide-multi-agent-orchestration-analysis
Link: [LangGraph AI Framework 2025: Complete Architecture Guide + Multi-Agent ...](https://latenode.com/blog/ai-frameworks-technical-infrastructure/langgraph-multi-agent-orchestration/langgraph-ai-framework-2025-complete-architecture-guide-multi-agent-orchestration-analysis)
Snippet: LangGraph's standout feature is its ability to maintain shared, persistent states across workflows, allowing dynamic adjustments based on runtime conditions. For example, in a document review system, agents can analyze text, flag issues, and pause for human feedback while retaining all prior context.

Source 3: Real-World Applications and Case Studies with LangGraph: From ... - Medium
URL: https://medium.com/@garima_yadav/real-world-applications-and-case-studies-with-langgraph-from-theory-to-practice-7a6ffd2e8e1b
Link: [Real-World Applications and Case Studies with LangGraph: From ... - Medium](https://medium.com/@garima_yadav/real-world-applications-and-case-studies-with-langgraph-from-theory-to-practice-7a6ffd2e8e1b)
Snippet: Introduction After exploring the fundamentals and advanced features of Langgraph , let's dive into practical implementations through real-world case studies.


---
**User:**

Research step: Collaborate with the LangGraph development team to discuss and prioritize future features based on your insights and predictions.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: Langgraph (LangGraph UserGroup) - Hugging Face
URL: https://huggingface.co/Langgraph
Link: [Langgraph (LangGraph UserGroup) - Hugging Face](https://huggingface.co/Langgraph)
Snippet: Mar 14, 2025 · We encourage you to join the LangGraph User Group to connect with fellow developers , ask questions, share your projects, and contribute to the LangGraph ecosystem.

Source 2: Build reliable, stateful AI systems, without giving up control
URL: https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute/
Link: [Build reliable, stateful AI systems, without giving up control](https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute/)
Snippet: Set up LangSmith for LangGraph development . Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects.

Source 3: Guide to Building AI Agents with LangGraph
URL: https://www.ema.co/additional-blogs/addition-blogs/building-ai-agents-langgraph
Link: [Guide to Building AI Agents with LangGraph](https://www.ema.co/additional-blogs/addition-blogs/building-ai-agents-langgraph)
Snippet: Jul 14, 2025 · In this blog, you'll get a clear, step-by-step guide to building AI agents with LangGraph. We'll cover what LangGraph is, its top features, the ... Missing: resources gain solid


---
**User:**

Research step: Continuously monitor updates and advancements in the field to stay informed about new features and improvements.
Top retrieved snippets (cite these URLs inline when you use them):
Source 1: example_langgraph_agents.ipynb - Colab
URL: https://colab.research.google.com/github/langfuse/langfuse-docs/blob/main/cookbook/example_langgraph_agents.ipynb
Link: [example_langgraph_agents.ipynb - Colab](https://colab.research.google.com/github/langfuse/langfuse-docs/blob/main/cookbook/example_langgraph_agents.ipynb)
Snippet: In this tutorial, we will learn how to monitor the internal steps (traces) of LangGraph agents and evaluate its performance using Langfuse and Hugging Face Datasets. This guide covers online and offline evaluation metrics used by teams to bring agents to production fast and reliably. To learn more about evaluation strategies, check out our blog post. Why AI agent Evaluation is important ...

Source 2: LangGraph vs n8n: Choosing the Right Framework for Agentic AI
URL: https://www.zenml.io/blog/langgraph-vs-n8n
Link: [LangGraph vs n8n: Choosing the Right Framework for Agentic AI](https://www.zenml.io/blog/langgraph-vs-n8n)
Snippet: Aug 1, 2025 · Compare LangGraph vs n8n for building AI agents in 2025. Updated with LangGraph 1.0 stable release and n8n's new unlimited workflow pricing. Missing: advancements stay informed

Source 3: LangGraph for Healthcare: A Comprehensive Technical Guide
URL: https://levelup.gitconnected.com/langgraph-for-healthcare-a-comprehensive-technical-guide-e6038b06c108
Link: [LangGraph for Healthcare: A Comprehensive Technical Guide](https://levelup.gitconnected.com/langgraph-for-healthcare-a-comprehensive-technical-guide-e6038b06c108)
Snippet: Nov 10, 2024 · Key Features. Real-time Analysis: Continuously monitors and analyzes hospital operations; Predictive Capabilities: Anticipates potential ... Missing: stay informed


---
**User:**

Now answer the original user question.
Reminder: you MUST follow the 3-step structure and inline-cite URLs from the snippets.
Question: Can you help me learn more about LangGraph? Specifically, can you predict the future possible features?



</blockquote>

</details>

</blockquote>

</details>

All Done! (This is also exactly what you'll want for Part 4.)


<hr><br>

## **Part 4:** Accumulating Your Reasoning Traces

Depending on the structure of your system, the last requirement may be trivial or might take a bit of extra effort. Please aggregate the answers to 8 diverse and reasonable questions, while also accumulating the trace (i.e. the "reasoning", projected to an understandable format). 

This output will be evaluated by an LLM to assess whether the response seems to exhibit reasonable behavior (reasoning makes sense, final output addresses question, sources are cited, etc).

In [29]:
## TODO: Aggregate 8 question-trace-answer triples. 
assert len(question_list) == 8, "question_list must have 8 questions"
assert len(answer_list) == 8, "answer_list must have 8 answers"
assert len(input_msgs_list) == 8, "input_msgs_list must have 8 traces"

def extract_trace(msgs):
    """
    msgs: list of (role, content)
    Your trace_messages are: [ (user, original question), (user, step1...), ..., (user, final_question...) ]
    For a clean 'trace', we drop the original question and the final "answer the question" prompt.
    """
    if not msgs:
        return []
    if len(msgs) == 1:
        return []
    if len(msgs) == 2:
        return [msgs[1]]
    return msgs[1:-1]

submission = []
for i in range(8):
    msgs = input_msgs_list[i].get("messages", [])
    submission.append({
        "question": question_list[i],
        "answer": answer_list[i],
        "trace": extract_trace(msgs),
    })

# Optional quick sanity check / preview
print("Built submission entries:", len(submission))
print("First entry keys:", submission[0].keys())
display(Markdown("**Preview entry 1**\n\n" + str(submission[0])[:1500]))

Built submission entries: 8
First entry keys: dict_keys(['question', 'answer', 'trace'])


**Preview entry 1**

{'question': 'Can you help me learn more about LangGraph? Specifically, can you tell me about Memory Management?', 'answer': 'Step 1: Core Concept(s)\n\nMemory management is a vital aspect of LangGraph, as it ensures the efficient allocation and deallocation of memory resources, which is crucial for building intelligent and context-aware applications. LangGraph, like other memory management systems, employs various strategies to manage memory, ensuring that memory is used optimally without causing performance issues. (Source 3: https://github.com/FareedKhan-dev/langgraph-long-memory)\n\nStep 2: Concrete Example\n\nLet\'s consider the concept of memory management in LangGraph more closely. One of the key features of LangGraph is its Store feature, which manages how memory is saved, retrieved, and updated, depending on where you are running your project. For example, if you are running LangGraph in-memory, the Store feature helps ensure that memory is allocated and deallocated efficiently. (Source 1: https://github.com/FareedKhan-dev/langgraph-long-memory)\n\nSource 2: No snippet provided for this step, but the main idea is to illustrate how the Store feature works in different environments.\n\nStep 3: How to Verify / Experiment / Apply\n\nTo verify or experiment with memory management in LangGraph, you can explore its official documentation and guides, such as the Comprehensive Guide: Long-Term Agentic Memory With LangGraph (Source 2: https://medium.com/@anil.jain.baba/long-te

<hr>
<br>

## **Part 5:** Running The Assessment

To assess your submission, run the following cells to save your results and the one after to query the assessment runner.

**Follow the instructions and make sure it all passes.**

In [30]:
import requests

## Send the submission over to the assessment runner
response = requests.post(
    "http://docker_router:8070/run_assessment", 
    json={"submission": submission, "model_specs": {"model": "nvidia/nemotron-3-nano-30b-a3b", "base_url": "http://llm_client:9000/v1"}},
)

response.raise_for_status()
try:
    response_dict = response.json()
except:
    response_dict = response.__dict__
display(Markdown(f"<h2>Assessment Response</h2>" + html_preview(response_dict, ["messages", "result"])))

<h2>Assessment Response</h2><ul><li><b>score</b>: 82
</li><li>
<b>passed</b>: True
</li><li>
<details open>

<summary><b>messages</b></summary>

<ol><li><details open>

<summary><b>[Case 0] Score:...</b></summary>

[Case 0] Score: 5/10 - The answer identifies why memory management matters in LangGraph, mentions the Store feature and suggests checking the official docs and a Reddit tutorial. It provides three logical steps, which meets the minimum requirement, but the steps are superficial. The 'Concrete Example' section promises code but delivers none, and Step 2 relies on a placeholder statement about the Store feature without any implementation details. Additionally, the response repeats source citations and uses generic phrasing like 'Source 2: No snippet provided', indicating placeholder content. To improve, include a short, annotated code snippet that shows how to bind a memory saver, explain allocation versus deallocation clearly, and reference at least three distinct insights from the retrieved material. Overall, the answer is on the right track but lacks depth and concrete illustration, so it falls around a mid‑range score.

</details>
</li><li>
<details open>

<summary><b>[Case 1] Score:...</b></summary>

[Case 1] Score: 5/10 - The response correctly identifies Pregel as the runtime engine of LangGraph and cites several sources, which shows an effort to ground the answer in external material. It also attempts to illustrate usage with a code snippet, which adds a concrete element. However, the answer falls short in several important ways: (1) the logical steps are only superficially distinct and do not build new insights—each step largely repeats the same information without deeper analysis; (2) the provided code contains placeholder identifiers and typographical errors (e.g., `langchain_chan.LanglChainChan`, `CADLActor`) that make it non‑executable and suggest a scaffold rather than a fully‑developed example; (3) citations are occasionally duplicated or mis‑referenced, and the inline‑citation format is inconsistent; (4) the answer does not explore Pregel’s query language, data model, or practical tips for optimizing queries, which were part of the research trace; (5) the explanation remains fairly high‑level and does not connect the cited sources to actionable knowledge or novel connections. To improve, the answer should (a) separate the explanation into at least three clearly different logical sections (e.g., architectural role, execution model, and practical optimization), (b) provide a correct, minimal runnable snippet that demonstrates state propagation across supersteps, (c) integrate citations to highlight how each source contributes a unique insight, and (d) adopt a conversational tone that walks the reader through the concepts step‑by‑step. Overall, the answer is on the right track but needs more depth, accuracy, and original synthesis to earn a higher score.

</details>
</li><li>
<details open>

<summary><b>[Case 2] Score:...</b></summary>

[Case 2] Score: 5/10 - Your answer correctly identifies three distinct steps—understanding the core concept of subgraphs, providing a concrete example with induced subgraphs, and describing how to verify or experiment with them. This meets the structural requirement of at least three logical steps. However, the response falls short in a few important areas: (1) it does not embed inline citations for the sources it references, violating the instruction to cite URLs directly when using content; (2) the explanation is rather superficial and repeats generic statements without deeper insights or concrete LangGraph code examples; (3) the discussion of applications such as graph coloring is broad and lacks specific guidance on how to implement these tasks within LangGraph. To improve, incorporate explicit inline citations (e.g., [Source 2]) wherever you draw from the retrieved snippets, add a brief code snippet or workflow diagram illustrating subgraph creation, and expand on at least one concrete use case (e.g., using a subgraph for modular routing in a multi‑agent system). Overall, the answer is on the right track but needs richer, source‑backed detail and more actionable information.

</details>
</li><li>
<details open>

<summary><b>[Case 3] Score:...</b></summary>

[Case 3] Score: 5/10 - The response does explain what full‑duplex communication is and correctly cites a few sources, but it falls short of the request. The question asked specifically about LangGraph, yet the answer never mentions LangGraph or ties the concept to that framework, leaving the core query unanswered. The reasoning is organized into three steps, which meets the minimum of distinct logical steps, but the steps are superficial and repeat information from the retrieved snippets without adding new synthesis or insight. Additionally, some citations are incomplete or point to placeholder URLs, and the answer repeats large blocks of snippet text without proper integration. To improve, the answer should explicitly connect full‑duplex communication to LangGraph (e.g., how LangGraph’s architecture may use full‑duplex concepts), provide clearer citations, and expand on the benefits and applications with more original analysis.

</details>
</li><li>
<details open>

<summary><b>[Case 4] Score:...</b></summary>

[Case 4] Score: 4/10 - The answer correctly identifies three logical steps—core concept, concrete example, and verification—but it falls short in several important ways. First, it uses placeholder citations instead of inline URL references, which violates the requirement to cite sources directly where they are used. The same source is repeated multiple times without adding new insight, making the explanation feel redundant and scaffold‑like rather than conversational. Additionally, the response stops at a high‑level summary and does not discuss key productionalization topics such as model versioning, containerization, or orchestration, so it lacks the depth needed for a thorough understanding. To improve, embed the URLs inline after each claim, expand each step with specific best‑practice details (e.g., using MLflow for versioning, Docker for container images, Kubernetes for orchestration), and adopt a more natural, flowing narrative that guides the reader through a practical workflow.

</details>
</li><li>
[Case 4] Below threshold (need 5+)
</li><li>
<details open>

<summary><b>[Case 5] Score:...</b></summary>

[Case 5] Score: 5/10 - The response tries to break the explanation into discrete steps, which shows an effort at structured reasoning. It identifies a force‑directed layout, gives a concrete word‑graph example, and suggests ways to verify the layout. However, the citations are either generic or do not correspond to the provided sources, and several statements (e.g., “Frumdlich's layout”) appear fabricated. The answer lacks depth about LangGraph’s actual visualization pipeline, repeats similar source references without new insight, and contains placeholder‑like URLs that do not support the claims. Overall, it demonstrates basic logical structuring but falls short on thoroughness, accuracy, and genuine new knowledge.

</details>
</li><li>
<details open>

<summary><b>[Case 6] Score:...</b></summary>

[Case 6] Score: 3/10 - The answer correctly cites a few sources and provides a very basic OCR snippet, which is a modest strength. However, it falls short in several key areas: (1) it misstates that Tesseract is “built on top of Language Detection for Python (LangDetect‑PY)”, which is factually incorrect; (2) it does not address LangGraph at all – the question explicitly asks for a LangGraph‑centric example, yet the response stays within a generic OCR workflow; (3) the three steps are repetitive scaffold‑like headings that add little new insight and do not demonstrate deeper understanding of how OCR can be integrated into a LangGraph graph (e.g., using image‑processing nodes, streaming events, or chaining with other agents); (4) the language is overly terse and lacks a conversational tone, and the code example contains errors (e.g., using `img.read()` on a Pillow Image object, which is unnecessary). To improve, the answer should explain how an image can be fed into a LangGraph node, show a minimal graph that runs OCR, include proper error handling, and connect the extracted text back into the graph’s state, while citing sources inline and keeping the discussion conversational.

</details>
</li><li>
[Case 6] Below threshold (need 5+)
</li><li>
<details open>

<summary><b>[Case 7] Score:...</b></summary>

[Case 7] Score: 4/10 - The answer attempts to follow the requested structure—covering a core concept, providing a concrete example, and describing verification/application—but it falls short on several fronts. First, many citations are incomplete or mismatched (e.g., URLs and snippet links do not correspond to the quoted text), and some referenced sources are not actually used to support the predictions. Second, the three logical steps are present but are fairly generic; they do not deliver deep, novel insights into future LangGraph features, nor do they convincingly tie predictions to observed trends or community feedback. The ‘Concrete Example’ introduces a vague NLP integration scenario (e.g., badge‑grade badminton tournaments) that adds little substantive value. Additionally, the answer repeats boilerplate statements about future features without demonstrating a rigorous analytical process. To improve, the response should: (1) cite sources accurately with correct inline links, (2) build each step on distinct, well‑explained reasoning, and (3) expand on concrete, anticipatable features grounded in the retrieved material. Overall, the answer is weak on thoroughness and new insight, meriting a modest score.

Score: 4

</details>
</li><li>
[Case 7] Below threshold (need 5+)</li></ol>

</details>
</li><li>
<b>exceptions</b>: []
</li><li>
<b>model_specs</b>: {'model': 'nvidia/nemotron-3-nano-30b-a3b', 'base_url': 'http://llm_client:9000/v1'}
</li><li>
<b>registry_summary</b>: {'total': 8, 'passed': 5, 'failed': 3}
</li><li>
<b>result</b>: Congratulations! You passed!
</li><li>
<b>visualization</b>: /dli/outputs/assessment_traces.html</li></ul>

In [31]:
from IPython.display import display, Markdown, HTML, IFrame
display(IFrame("assessment_outputs/assessment_traces.html", width="100%", height=680))

<br>

If you passed the assessment, please return to the course page (shown below) and click the **"ASSESS TASK"** button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">

<hr>
<br>

## **Part 6:** Wrapping Up

### <font color="#76b900">**Congratulations On Completing The Course!!**</font>

Before concluding the course, we highly recommend downloading the course material for later reference, and checking over the **"Next Steps"** and **Feedback** sections of the course. **We appreciate you taking the time to go through the course, and look forward to seeing you again for the next courses in the series!**

<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>